# Building Abstractions with Procedures

## Exercise 1.1
Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented
```racket
10
(+ 5 3 4)
(- 9 1)
(/ 6 2)
(+ (* 2 4) (- 4 6))
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
(= a b)
(if (and (> b a) (< b (* a b)))
    b
    a)
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
(+ 2 (if (> b a) b a))
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
```

In [38]:
10

10

In [39]:
(+ 5 3 4)

12

In [40]:
(- 9 1)

8

In [41]:
(/ 6 2)

3

In [42]:
(+ (* 2 4) (- 4 6))

6

In [43]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))

19

In [44]:
(= a b)

#f

In [45]:
(if (and (> b a) (< b (* a b)))
    b
    a)

4

In [46]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

In [47]:
(+ 2 (if (> b a) b a))

6

In [48]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

## Exercise 1.2
Translate the following expression into prefix form
$$
\frac{5+4+(2-(3-(6+\frac{4}{3})))}{3(6-2)(2-7)}
$$

In [49]:
(/ (+ 5 4 (- 2
             (- 3
                (+ 6 (/ 4 3)))))
   (* 3 (- 6 2) (- 2 7)))

-23/90

## Exercise 1.3
Define a procedure that takes three numbers as arguments and returns the sum of the squares of the two larger numbers.

In [50]:
(define (square x)
  (* x x))
(define (sum_of_squares a b)
  (+ (square a) (square b)))
(define (sum_square_of_two_larger x y z)
  (cond ((and (> x z) (> y z))
        (sum_of_squares x y))
        ((> x y)
         (sum_of_squares x z))
        (else (sum_of_squares y z))))

## Exercise 1.4
Observe that our model of evaluation allows for combinations whose operators are compound expressions. Use this observation to describe the behavior of the following procedure:
```racket
(define (a-plus-abs-b a b)
  ((if (> b 0) + -) a b))
```

When we evalute `(a-plus-abs-b a b)`, operator part `(if (> b 0) + -)` will be evaluted first. If `b` is positive, then `(+ a b)` will be evaluated. If `b` is negative, then `(- a b)` will be evaluated.

## Exercise 1.5
Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative-order evaluation or normal-order evaluation. He defines the following two procedures:
```racket
(define (p) (p))

(define (test x y)
  (if (= x 0)
      0
      y))
```
Then he evaluates the expression
```racket
(test 0 (p))
```
What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form `if` is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)

Applicative-order:</br>
The operator with `if` will be evalute first, then we get 0 directly.</br>
Normal-order:</br>
To substitute `(p)`, we get another `(p)` in `(test 0 (p))` , it's a infinite loop.</br>

## Exercise 1.6
Alyssa P. Hacker doesn't see why `if` needs to be provided as a special form. ``Why can't I just define it as an ordinary procedure in terms of `cond`?'' she asks. Alyssa's friend Eva Lu Ator claims this can indeed be done, and she defines a new version of `if`:

```racket
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))
```
Eva demonstrates the program for Alyssa:

```racket
(new-if (= 2 3) 0 5)
5
```

```racket
(new-if (= 1 1) 0 5)
0
```
Delighted, Alyssa uses `new-if` to rewrite the square-root program:

```racket
(define (sqrt-iter guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter (improve guess x)
                     x)))
```
What happens when Alyssa attempts to use this to compute square roots? Explain.

Using `new-if`, it takes long time without respon.

Becase `new-if` is a function. It must evaluate `sqrt-iter` in operand first, so it falls in to a infinite loop.But `if` is a special from, only if the `<predicate>` is `#f`,  `<alternative>` will be evaluted.

## Exercise 1.7
The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how `guess` changes from one iteration to the next and to stop when the change is a very small fraction of the guess. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers?

eg. For `(sqrt 0.0000000000001)` or `(sqrt 30864197530864197530864197530864197530864197530864197524691358024691358024691358024691358024691358024691358025)` the process dosen't work.
The improved program is below:

In [51]:
(define (improve guess x)
  (average guess (/ x guess)))
(define (average x y)
  (/ (+ x y) 2))
(define (good-enough? guess old-guess)
  (< (/ (abs (- old-guess guess)) 2) 0.001))
(define (sqrt x)
  (sqrt-iter 1.0 0.0 x))
(define (sqrt-iter guess old-guess x)
  (if (good-enough? guess old-guess)
          guess
          (sqrt-iter (improve guess x)
                     guess
                     x)))

## Exercise 1.8
Newton's method for cube roots is based on the fact that if y is an approximation to the cube root of x, then a better approximation is given by the value
$$\frac{x/y^2+2y}{3}$$
Use this formula to implement a cube-root procedure analogous to the square-root procedure. (In section 1.3.4 we will see how to implement Newton's method in general as an abstraction of these square-root and cube-root procedures.)

In [52]:
(define (improve guess x)
  (/ (+ (/ x (* guess guess)) (* 2 guess))
     3))
(define (good-enough? guess old-guess)
  (< (/ (abs (- old-guess guess)) 2) 0.001))
(define (cube-root x)
  (cube-root-iter 1.0 0.0 x))
(define (cube-root-iter guess old-guess x)
  (if (good-enough? guess old-guess)
          guess
          (cube-root-iter (improve guess x)
                     guess
                     x)))

## Exercise 1.9
Each of the following two procedures defines a method for adding two positive integers in terms of the procedures `inc`, which increments its argument by 1, and `dec`, which decrements its argument by 1.
```racket
(define (+ a b)
  (if (= a 0)
      b
      (inc (+ (dec a) b))))

(define (+ a b)
  (if (= a 0)
      b
      (+ (dec a) (inc b))))
```
Using the substitution model, illustrate the process generated by each procedure in evaluating `(+ 4 5)`. Are these processes iterative or recursive?

For the first process
```racket
(+ 4 5)
(inc (+ 3 5))
(inc (inc (+ 2 5)))
(inc (inc (inc (+ 1 5))))
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
(inc (inc (inc 6)))
(inc (inc 7))
(inc 8)
9
```
This is a recursive process.
```racket
(+ 4 5)
(+ 3 6)
(+ 2 7)
(+ 1 8)
(+ 0 9)
9
```
This is a iterative process.

## Exercise 1.10
The following procedure computes a mathematical function called Ackermann's function.
```racket
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))
```
What are the values of the following expressions?
```racket
(A 1 10)
(A 2 4)
(A 3 3)
```
Consider the following procedures, where A is the procedure defined above:
```racket
(define (f n) (A 0 n))
(define (g n) (A 1 n))
(define (h n) (A 2 n))
(define (k n) (* 5 n n))
```
Give concise mathematical definitions for the functions computed by the procedures `f`, `g`, and `h` for positive integer values of $n$. For example, `(k n)` computes $5n^2$.


In [53]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))

In [54]:
(A 1 10)

1024

In [55]:
(A 2 4)

65536

In [56]:
(A 3 3)

65536

`(f n)` computes $2n$</br>
`(g n)` computes $2^n$</br>
`(h n)` computes $2$ square $n - 1$ times and when $n = 0$, it's $0$

## Exercise 1.11
A function $f$ is defined by the rule that $f(n) = n$ if $n<3$ and $f(n) = f(n - 1) + 2f(n - 2) + 3f(n - 3)$ if $n> 3$. Write a procedure that computes $f$ by means of a recursive process. Write a procedure that computes $f$ by means of an iterative process.

In [57]:
; Recusive Process
(define (f n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        ((= n 2) 2)
        (else (+ (f (- n 1))
                 (* 2 (f (- n 2)))
                 (* 3 (f (- n 3)))))))

In [58]:
; Iterative Process
(define (f n)
  (define (iter a b c count)
    (if (= count 0)
        a
        (iter b c (+ (* 3 a) (* 2 b) c) (- count 1))))
  (iter 0 1 2 n))